In [ ]:
from surface_dynamics.all import Origami

In [ ]:
# The matrices S, S^(-1), and R generate the group SL2Z
S = Matrix(ZZ, [[1,1],[0,1]])
R = Matrix(ZZ,[[0,-1],[1,0]])

# The SL2Z_Action function takes an origami ori and a 2x2 matrix A with integer coefficients as arguments
# It tests whether A is an element of SL2Z, and if so it returns the image of ori under the action of A
# If A is not an element of SL2Z, it returns an error message
def SL2Z_Action(ori, A):
    
    # Tests whether A is in SL2Z
    if det(A) != 1:
        return "Error: The input matrix is not an element of SL2Z"
        
    # List that will hold generators that produce A when multiplied in the order they appear
    # Temporary matrix that we will update as we reduce A to the identity using generators
    generators = []
    A_temp = A
    
    # Reduce A_temp to the identity by multiplying on the left by S, S^(-1), and R repeatedly
    # Simultaneously add the inverse of the matrix that is being multiplied to the end of list of generators
    while A_temp[1][0] != 0:
        if abs(A_temp[0][0])>=abs(A_temp[1][0]):
            if sgn(A_temp[0][0]) == sgn(A_temp[1][0]):
                A_temp = S^(-1)*A_temp
                generators = generators + [S]
            else:
                A_temp = S*A_temp
                generators = generators + [S^(-1)]
        else:
            A_temp = R*A_temp
            generators = generators + [R,R,R]
    if A_temp[0][0] == 1:
        while A_temp[0][1] > 0:
            A_temp = S^(-1)*A_temp
            generators = generators + [S]
        while A_temp[0][1] < 0:
            A_temp = S*A_temp
            generators = generators + [S^(-1)]
    else:
        while A_temp[0][1] < 0:
            A_temp = S^(-1)*A_temp
            generators = generators + [S]
        while A_temp[0][1] > 0:
            A_temp = S*A_temp
            generators = generators + [S^(-1)]
        A_temp = R*R*A_temp
        generators = generators + [R,R]
    
    # Test to confirm that generators in list multiply to produce A
    # If not, return error message
    product = Matrix(ZZ,[[1,0],[0,1]])
    for generator in generators:
        product = product*generator
    if product != A:
        return "Error: Decomposition into product of generators has failed"
        
    # Origami that will be returned after the action of A is applied
    A_times_ori = ori
    
    # Apply generators in list in reverse order because A is multiplied on the left of ori
    for generator in reversed(generators):
        if generator == R:
            A_times_ori = A_times_ori.horizontal_symmetry().mirror()
            # print('R')
        elif generator == S:
            A_times_ori = A_times_ori.horizontal_twist(1)
            # print('S')
        else:
            A_times_ori = A_times_ori.horizontal_twist(-1)
            # print('S_inv')
            
    return A_times_ori

In [ ]:
# The test_For_Cylinders function tests whether an input origami ori has a convex presentation
# via the 1-clyinder direction condition. It tests first that ori is hyperelliptic, then it
# tests every element of ori's SL2Z orbit for a horizontal 1-cylinder. Returns True if condition 
# is satisfied and False otherwise
def test_For_Cylinders(ori):
    if ori.is_hyperelliptic():
        for A in ori.veech_group().coset_reps():
            A_inv = A^(-1) # Use inverse of A because coset_reps() gives right coset reps
            A_inv_times_ori = SL2Z_Action(ori, A_inv)
            if len(A_inv_times_ori.cylinder_decomposition()) == 1:
                return True
    return False

In [ ]:
# The origamis_In_S_n function returns a list of all unique origamis of area n in the genus H(2).
# It takes as its input a positive integer n. It uses the function H_2_origami_from_dimensions to
# Create origami objects based on certain parameters.
def origamis_In_S_n(n):
    listOfOrigamis = []
    for i in range(1,n):
        j = n-i
        for u_1 in range(1,i+1):
            if i%u_1 == 0:
                h_1 = i//u_1
                for u_2 in range(u_1+1,j+1):
                    if j%u_2 == 0:
                        h_2 = j//u_2
                        for t_1 in range(u_1):
                            for t_2 in range(u_2):
                                listOfOrigamis.append(H_2_origami_from_dimensions(u_1,h_1,t_1,u_2,h_2,t_2))
    return listOfOrigamis

In [ ]:
# The H_2_origami_from_dimensions creates an origami consisting of two parallelograms glued together
# with hights h_1 and h_2, widths u_1 and u_2, and twists t_1 and t_2. Every origami in H(2) has this form
def H_2_origami_from_dimensions(u_1,h_1,t_1,u_2,h_2,t_2):
    pi_r = ''
    pi_u = ''
    
    horiz_1 = ''
    horiz_2 = ''
    for k in range(h_2):
        horiz_2 += '('
        for l in range(1,u_2):
            horiz_2 += str(k*u_2+l)+','
        horiz_2 += str(k*u_2+u_2)+')'
    for k in range(h_1):
        horiz_1 += '('
        for l in range(1,u_1):
            horiz_1 += str(u_2*h_2+k*u_1+l)+','
        horiz_1 += str(u_2*h_2+k*u_1+u_1)+')'
    pi_r = horiz_2 + horiz_1
    
    ver_1 = ''
    ver_2 = ''
    glue_vals_1 = []
    glue_vals_2 = []
    for k in range(1,u_2+1):
        ver_2 += '('
        for l in range(h_2-1):
            ver_2 += str(l*u_2+k)+','
            if l==0:
                glue_vals_2.append(str(l*u_2+k))
        ver_2 += str((h_2-1)*u_2+k)+')'
        if h_2==1:
            glue_vals_2.append(str((h_2-1)*u_2+k))
    for k in range(1,u_1+1):
        ver_1 += '('
        for l in range(h_1-1):
            ver_1 += str(u_2*h_2+l*u_1+k)+','
            if l==0:
                glue_vals_1.append(str(u_2*h_2+l*u_1+k))
        ver_1 += str(u_2*h_2+(h_1-1)*u_1+k)+')'
        if h_1==1:
            glue_vals_1.append(str(u_2*h_2+(h_1-1)*u_1+k))
    pi_u = ver_1 + ver_2
 
    gluing_permutation = Permutation('(1)')
    for k in range(u_1):
        gluing_permutation *= Permutation('(' + glue_vals_1[k] + ',' + glue_vals_2[k] + ')')
    pi_u = (Permutation(pi_u)*gluing_permutation).cycle_string()
    
    glue_vals_1 = []
    glue_vals_2 = []
    if t_1>0:
        for k in range(1,u_1+1):
            if str(k) not in glue_vals_2:
                glue_vals_1.append(str(k))
                glue_val_2 = k-t_1
                if glue_val_2 < 1:
                    glue_val_2 += u_1
                glue_val_2 -= t_2
                if glue_val_2 < 1:
                    glue_val_2 += u_2
                glue_vals_2.append(str(glue_val_2))
    if t_2>0:
        for k in range(u_1+1,u_2+1):
            if str(k) not in glue_vals_2:
                glue_vals_1.append(str(k))
                glue_val_2 = k-t_2
                if glue_val_2 < 1:
                    glue_val_2 += u_2
                glue_vals_2.append(str(glue_val_2))

    for k in range(len(glue_vals_1)):
        if glue_vals_1[k] == glue_vals_2[k]:
            glue_vals_1[k] = 0
            glue_vals_2[k] = 0
    while 0 in glue_vals_1:
        glue_vals_1.remove(0)
    while 0 in glue_vals_2:
        glue_vals_2.remove(0)
        
#     print(glue_vals_1)
#     print(glue_vals_2)
    gluing_permutation = Permutation('(1)')
    for k in range(len(glue_vals_1)):
        gluing_permutation *= Permutation('(' + glue_vals_1[k] + ',' + glue_vals_2[k] + ')')
    pi_u = (Permutation(pi_u)*gluing_permutation).cycle_string()
    
    
    return Origami(pi_r,pi_u)

In [ ]:
H_2_origami_from_dimensions(3,4,2,5,2,1)

In [ ]:
origamis_In_S_n(4)

In [ ]:
for n in range(1,101):
    print("Origamis with area ",n," that do not have a one-cylinder direction:")
    for ori in origamis_In_S_n(n):
        if not test_For_Cylinders(ori):
            print(ori)